# SQL

## Conexión a PostgreSQL usando SQLAlchemy

In [57]:
import os
import pandas as pd
from typing import Union
from sqlalchemy import create_engine, text

# Reemplaza USER y PASSWORD del .env
engine = create_engine("sqlite:///data/sql-murder-mystery.db")

# Probar la conexión
try:
    connection = engine.connect()
    print("Conexión exitosa")
    connection.close()
except Exception as e:
    print(f"Error al conectar: {e}")


def ejecutar_query(query: str) -> Union[pd.DataFrame, str]:
    """
    Ejecuta una consulta SQL en la base de datos y devuelve un DataFrame.

    Parámetros:
    - query (str): La consulta SQL a ejecutar.

    Retorna:
    - Union[pd.DataFrame, pl.DataFrame, str]: Retorna un DataFrame en pandas o polars dependiendo del parámetro 'devolver_polars',
      o un mensaje de éxito/error si la operación no es un SELECT.
    """
    
    with engine.connect() as connection:
        try:
            result = connection.execute(text(query))
            
            # Determinar si es una consulta SELECT para devolver datos
            if result.returns_rows:
                rows = result.fetchall()
                columns = result.keys()

                # Convertir filas en un diccionario de listas para crear correctamente el DataFrame en polars
                data_dict = {col: [row[idx] for row in rows] for idx, col in enumerate(columns)}

                # Verificación de coincidencia de columnas y filas
                return pd.DataFrame(data_dict)
            else:
                connection.commit()
                return f"Query ejecutada correctamente: {result.rowcount} filas afectadas."
        except Exception as e:
            return f"Error al ejecutar la query: {e}"


Conexión exitosa


In [58]:
pd.set_option ('display.max_colwidth', True)

query=  """ SELECT *
                FROM crime_scene_report
                WHERE type =  'murder'
                AND city = 'SQL City'
                AND date = '20180115'
"""
ejecutar_query(query=query)

,date,type,description,city
0,20180115,murder,"Security footage shows that there were 2 witnesses. The first witness lives at the last house on ""Northwestern Dr"". The second witness, named Annabel, lives somewhere on ""Franklin Ave"".",SQL City


In [59]:
query = """SELECT *
        FROM person """

ejecutar_query(query=query)

,id,name,license_id,address_number,address_street_name,ssn
0,10000,Christoper Peteuil,993845,624,Bankhall Ave,747714076
1,10007,Kourtney Calderwood,861794,2791,Gustavus Blvd,477972044
2,10010,Muoi Cary,385336,741,Northwestern Dr,828638512
3,10016,Era Moselle,431897,1987,Wood Glade St,614621061
4,10025,Trena Hornby,550890,276,Daws Hill Way,223877684
...,...,...,...,...,...,...
10006,99936,Luba Benser,274427,680,Carnage Blvd,685095054
10007,99941,Roxana Mckimley,975942,1613,Gate St,512136801
10008,99965,Cherie Zeimantz,287627,3661,The Water Ave,362877324
10009,99982,Allen Cruse,251350,3126,N Jean Dr,348734531


In [60]:
query = """SELECT * 
    FROM person 
    WHERE address_street_name = 'Northwestern Dr' 
        OR address_street_name = 'Franklin Ave'
"""
ejecutar_query(query=query)

,id,name,license_id,address_number,address_street_name,ssn
0,10010,Muoi Cary,385336,741,Northwestern Dr,828638512
1,12207,Wilmer Wolever,509484,139,Franklin Ave,636825374
2,12711,Norman Apolito,667757,599,Northwestern Dr,778264744
3,14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949
4,15171,Weldon Penso,336999,311,Northwestern Dr,131379495
...,...,...,...,...,...,...
90,96595,Coretta Cubie,303645,3631,Northwestern Dr,378403829
91,97913,Cameron Dilick,971988,2954,Franklin Ave,665147939
92,98593,Jonah Toner,578982,2947,Northwestern Dr,745853548
93,98744,Jordan Myntti,256116,3104,Franklin Ave,876030104


In [61]:
query = """SELECT * 
    FROM person 
    WHERE address_street_name = 'Northwestern Dr' 
    ORDER BY address_number DESC
"""
ejecutar_query(query=query).head(1)

,id,name,license_id,address_number,address_street_name,ssn
0,14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949


In [62]:
query = """SELECT *
    FROM person 
    WHERE address_street_name = 'Franklin Ave'
    AND name LIKE 'Annabel%'
"""
ejecutar_query(query=query)

,id,name,license_id,address_number,address_street_name,ssn
0,16371,Annabel Miller,490173,103,Franklin Ave,318771143


In [63]:
query = """ SELECT * 
    FROM interview
    WHERE person_id = '14887' 
    OR person_id = '16371' 
"""
ejecutar_query(query=query)

,person_id,transcript
0,14887,"I heard a gunshot and then saw a man run out. He had a ""Get Fit Now Gym"" bag. The membership number on the bag started with ""48Z"". Only gold members have those bags. The man got into a car with a plate that included ""H42W""."
1,16371,"I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th."


In [64]:
#consulto la tabla del gimnasio para ver cuanta gente hay y la pinta que tiene
query ="""SELECT *
    FROM get_fit_now_member"""

ejecutar_query(query=query)

,id,person_id,name,membership_start_date,membership_status
0,NL318,65076,Everette Koepke,20170926,gold
1,AOE21,39426,Noe Locascio,20171005,regular
2,2PN28,63823,Jeromy Heitschmidt,20180215,silver
3,0YJ24,80651,Waneta Wellard,20171206,gold
4,3A08L,32858,Mei Bianchin,20170401,silver
...,...,...,...,...,...
179,2V137,41693,Wendell Dulany,20171219,silver
180,4KB72,79110,Emile Hege,20170522,regular
181,48Z7A,28819,Joe Germuska,20160305,gold
182,48Z55,67318,Jeremy Bowers,20160101,gold


Filtro las condiciones que indicó el testigo Morty Shappiro

In [65]:
query ="""SELECT *
    FROM get_fit_now_member
    WHERE membership_status = 'gold'
    AND id LIKE '48Z%'
    """

ejecutar_query(query=query)

,id,person_id,name,membership_start_date,membership_status
0,48Z7A,28819,Joe Germuska,20160305,gold
1,48Z55,67318,Jeremy Bowers,20160101,gold


In [66]:
query= """SELECT person.name, drivers_license.plate_number
    FROM person, drivers_license
    WHERE person.license_id = drivers_license.id
    AND person.id = '28819' OR person.id= '67318'

    """

ejecutar_query(query=query)

,name,plate_number
0,Jeremy Bowers,P24L4U
1,Jeremy Bowers,XF02T6
2,Jeremy Bowers,VKY5KR
3,Jeremy Bowers,Y5NZ08
4,Jeremy Bowers,5162Z1
...,...,...
10002,Jeremy Bowers,5L0ZI4
10003,Jeremy Bowers,1B8QN8
10004,Jeremy Bowers,1684K3
10005,Jeremy Bowers,F8F64H


Parece que solo aparece Jeremy Bowers (que tiene muchos coches, por otro lado)... voy a ver por qué no aparece Joe Germuska

In [67]:
query= """SELECT person.id AS 'person.id', person.license_id, drivers_license.plate_number
    FROM person, drivers_license
    WHERE person.license_id = drivers_license.id
    AND person.id = '28819'
    """

ejecutar_query(query=query)

,person.id,license_id,plate_number


Me sale vacio, como no me fio mucho de mi mismo, pruebo haciéndolo en pasos...

In [68]:
query = "SELECT * FROM person WHERE id = '28819'"
ejecutar_query(query=query)

,id,name,license_id,address_number,address_street_name,ssn
0,28819,Joe Germuska,173289,111,Fisk Rd,138909730


In [69]:
query = "SELECT * FROM drivers_license WHERE id= '173289'"
ejecutar_query(query=query)

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model


Así he comprobado que Joe Germuska tiene licencia de conducir, pero no está en la tabla (quizá no tenga coches, o licencia caducada...).

Veamos ahora si Jeremy Bowers tiene algún coche con esos números en la matrícula

In [81]:
query= """SELECT person.name, drivers_license.plate_number
    FROM person, drivers_license
    WHERE person.license_id = drivers_license.id
    AND person.id= '67318'
    AND plate_number LIKE '%H42W%'

    """

ejecutar_query(query=query)

,name,plate_number
0,Jeremy Bowers,0H42W2


También podemos hacerlo por pasos. 
1. primero veamos cuantos coches hay con matrícula con H42W...

In [71]:
query= """SELECT *
    FROM drivers_license
    WHERE plate_number LIKE '%H42W%'
    """

ejecutar_query(query=query)

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,183779,21,65,blue,blonde,female,H42W0X,Toyota,Prius
1,423327,30,70,brown,brown,male,0H42W2,Chevrolet,Spark LS
2,664760,21,71,black,black,male,4H42WR,Nissan,Altima


2. veamos sus propietarios, a nombre de quién están...

In [77]:
query= """SELECT person.id AS 'Person.id', person.name, drivers_license.plate_number
    FROM person, drivers_license
    WHERE person.license_id = drivers_license.id
    AND drivers_license.plate_number LIKE '%H42W%'

    """

ejecutar_query(query=query)

,Person.id,name,plate_number
0,51739,Tushar Chandra,4H42WR
1,67318,Jeremy Bowers,0H42W2
2,78193,Maxine Whitely,H42W0X


In [76]:
query= """SELECT *
    FROM person
    WHERE license_id = '183779'
    OR license_id = '423327'
    OR license_id = '664760'
    """

ejecutar_query(query=query)

,id,name,license_id,address_number,address_street_name,ssn
0,51739,Tushar Chandra,664760,312,Phi St,137882671
1,67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279
2,78193,Maxine Whitely,183779,110,Fisk Rd,137882671


Pues ya confirmamos que Jeremy Bowers sí tiene el coche con la matrícula vista por el testigo.

### Así pues, parece que el asesino es Jeremy Bowers.

#### La parte extra de la web
Congrats, you found the murderer! But wait, there's more... If you think you're up for a challenge, try querying the interview transcript of the murderer to find the real villain behind this crime. If you feel especially confident in your SQL skills, try to complete this final step with no more than 2 queries. Use this same INSERT statement with your new suspect to check your answer.

In [89]:
pd.set_option ('display.max_colwidth', False)

query=  """ SELECT *
            FROM interview
            WHERE person_id =  '67318'
"""
ejecutar_query(query=query)

,person_id,transcript
0,67318,"I was hired by a woman with a lot of money. I don't know her name but I know she's around 5'5"" (65"") or 5'7"" (67""). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017.\n"


Así que la inductora del asesinato es una mujer que contrató a Jeremy Bowers. Vamos a buscarla...

In [90]:
query=  """ SELECT *
            FROM drivers_license
            WHERE hair_color =  'red'
"""
ejecutar_query(query=query)

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,100280,72,57,brown,red,male,P24L4U,Acura,MDX
1,101494,48,55,blue,red,female,81X1N7,Kia,Sportage
2,102950,56,78,brown,red,male,4BR8A0,Honda,Insight
3,103197,38,73,amber,red,female,273D0J,Saab,9-7X
4,106868,58,73,black,red,male,YF2710,Jeep,Grand Cherokee
...,...,...,...,...,...,...,...,...,...
1262,993748,44,66,amber,red,female,GONCC5,Honda,Fit
1263,994453,70,75,brown,red,female,O5377P,Hyundai,Tiburon
1264,996577,77,77,green,red,female,L0B62F,Chevrolet,Silverado 1500
1265,998194,59,65,black,red,female,U85D0X,Mitsubishi,Lancer Evolution


No se suficiente con ese filto de pelirroja, lo afino poco a poco...

In [ ]:
# ESTA SERÍA LA QUERY 1
query=  """ SELECT *
            FROM drivers_license
            WHERE hair_color =  'red'
            AND height BETWEEN 65 AND 67
            AND car_make = 'Tesla'
            AND car_model = 'Model S'
"""
ejecutar_query(query=query)

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,202298,68,66,green,red,female,500123,Tesla,Model S
1,291182,65,66,blue,red,female,08CM64,Tesla,Model S
2,918773,48,65,black,red,female,917UU3,Tesla,Model S


In [116]:
#LA MISMA QUERY, INCORPORANDO EL PERSON ID QUE NECESITAREMOS
# ESTA SERÍA LA QUERY 1
query=  """ SELECT person.id, person.license_id, gender, height, hair_color, car_make, car_model
            FROM person 
            JOIN drivers_license
            ON person.license_id = drivers_license.id
            WHERE hair_color = 'red'
            AND height BETWEEN 65 AND 67
            AND car_make = 'Tesla'
            AND car_model = 'Model S'
"""
ejecutar_query(query=query)

,id,license_id,gender,height,hair_color,car_make,car_model
0,78881,918773,female,65,red,Tesla,Model S
1,90700,291182,female,66,red,Tesla,Model S
2,99716,202298,female,66,red,Tesla,Model S


In [118]:
# ESTA SERÍA LA QUERY 2
query=  """ SELECT *
            FROM facebook_event_checkin
            WHERE date BETWEEN 20171201 AND 20171231
            AND event_name LIKE '%SQL%'

"""
ejecutar_query(query=query).tail()

,person_id,event_id,event_name,date
16,24556,1143,SQL Symphony Concert,20171224
17,99716,1143,SQL Symphony Concert,20171206
18,99716,1143,SQL Symphony Concert,20171212
19,99716,1143,SQL Symphony Concert,20171229
20,67318,1143,SQL Symphony Concert,20171206


In [134]:
#aunando ambas consultas

query=  """ SELECT person.id, person.name, person.license_id, gender, height, hair_color, car_make, car_model, event_name,date
            FROM person 

            JOIN drivers_license
            ON person.license_id = drivers_license.id
            JOIN facebook_event_checkin
            ON person.id = facebook_event_checkin.person_id

            WHERE date BETWEEN 20171201 AND 20171231
            AND event_name LIKE '%SQL%'

            AND hair_color = 'red'
            AND height BETWEEN 65 AND 67
            AND car_make = 'Tesla'
            AND car_model = 'Model S'
            
"""
ejecutar_query(query=query)

,id,name,license_id,gender,height,hair_color,car_make,car_model,event_name,date
0,99716,Miranda Priestly,202298,female,66,red,Tesla,Model S,SQL Symphony Concert,20171206
1,99716,Miranda Priestly,202298,female,66,red,Tesla,Model S,SQL Symphony Concert,20171212
2,99716,Miranda Priestly,202298,female,66,red,Tesla,Model S,SQL Symphony Concert,20171229


Aquí está el resultado, cumpliendo todas las pistas dadas por el asesino

Congrats, you found the brains behind the murder! Everyone in SQL City hails you as the greatest SQL detective of all time. Time to break out the champagne!